In [ ]:
import numpy as np

def nonlinear_transform(x1, x2): #function to perform the nonlinear transform
  return np.array([1, x1, x2, x1**2, x2**2, x1 * x2, abs(x1 - x2), abs(x1 + x2)])

def parse_data(filename): #function to parse the data for each file
  data = []
  labels = []
  with open(filename, 'r') as file: #open the file
    for line in file: #for each line of the file get the data x1, x2, y
      x1, x2, y = map(float, line.strip().split())
      transformed_x = nonlinear_transform(x1, x2) #transform the data
      data.append(transformed_x) #append the transformed data to data
      labels.append(y) #get the labels
  return np.array(data), np.array(labels)

def linear_regression(X, y): #perform linear regression to find weights
  return np.linalg.pinv(X) @ y

def misclassified(y_true, y_pred): #calculate classification error
  return np.mean(y_true != np.sign(y_pred))

data, labels = parse_data("training.txt") #load in sample data

X_train, y_train = data[:25], labels[:25]
X_val, y_val = data[25:], labels[25:]

best_k = None
lowest_error = float('inf')

for k in range(3, 8):
  X_train_k = X_train[:, :k+1] #use only features up to k
  X_val_k = X_val[:, :k+1]

  weights = linear_regression(X_train_k, y_train) #train model on training set
  y_val_pred = X_val_k @ weights #predict on validation set

  error = misclassified(y_val, y_val_pred) #calculate classification error
  if error < lowest_error:
    lowest_error = error
    best_k = k #update best model if error is lower

print(f'k = {best_k}')

k = 6


In [ ]:
X_test, y_test = parse_data("testing.txt") #load out of sample data

out_sample_errors = {} #dictionary to store error for each model

for k in range(3, 8):
    X_train_k = X_train[:, :k+1]
    X_test_k = X_test[:, :k+1] #full out-of-sample data for testing

    weights = linear_regression(X_train_k, y_train) #train model on the training data
    y_test_pred = X_test_k @ weights #predict on the out-of-sample data

    out_sample_errors[k] = misclassified(y_test, y_test_pred) #calculate the out-of-sample error

best_k_out_sample = min(out_sample_errors, key=out_sample_errors.get)
print(f"k = {best_k_out_sample}")
print(f'error = {min(out_sample_errors.values())}')

k = 7
error = 0.072


In [ ]:
data, labels = parse_data("training.txt") #load in sample data

validation_errors = {} #dictionary to store validation errors for each model

X_val, y_val = data[:25], labels[:25] #first 25 for validation
X_train, y_train = data[25:], labels[25:] #last 10 for training

for k in range(3, 8):
  X_train_k = X_train[:, :k+1]
  X_val_k = X_val[:, :k+1]

  weights = linear_regression(X_train_k, y_train) #train on training set
  y_val_pred = X_val_k @ weights #prediction on validation set

  validation_errors[k] = misclassified(y_val, y_val_pred) #calc classification error

best_k = min(validation_errors, key=validation_errors.get)
print(f'k = {best_k}')

k = 6


In [ ]:
X_test, y_test = parse_data("testing.txt") #load out of sample data

out_sample_errors = {} #dictionary to store error for each model

for k in range(3, 8):
    X_train_k = X_train[:, :k+1]
    X_test_k = X_test[:, :k+1] #full out-of-sample data for testing

    weights = linear_regression(X_train_k, y_train) #train model on the training data
    y_test_pred = X_test_k @ weights #predict on the out-of-sample data

    out_sample_errors[k] = misclassified(y_test, y_test_pred) #calculate the out-of-sample error
best_k_out_sample = min(out_sample_errors, key=out_sample_errors.get)
print(f"k = {best_k_out_sample}")
print(f'error = {min(out_sample_errors.values())}')

k = 6
error = 0.192


In [ ]:
num_samples = 10**6 #number of simulations

#generate random samples for e1 and e2
e1 = np.random.uniform(0, 1, num_samples)
e2 = np.random.uniform(0, 1, num_samples)

#calculate min(e1, e2) for each pair
e = np.minimum(e1, e2)

#calculate expected values
E_e1 = np.mean(e1)
E_e2 = np.mean(e2)
E_e = np.mean(e)

print(f"Expected value of e1: {E_e1:.4f}")
print(f"Expected value of e2: {E_e2:.4f}")
print(f"Expected value of e: {E_e:.4f}")

Expected value of e1: 0.5005
Expected value of e2: 0.5000
Expected value of e: 0.3337


In [ ]:
import numpy as np
from cvxopt import matrix, solvers
import random

def generate_target_function():
    #generate two random points in [-1, 1] x [-1, 1]
    p1 = np.random.uniform(-1, 1, 2)
    p2 = np.random.uniform(-1, 1, 2)

    #define the line passing through these points
    return p1, p2

def generate_data(N, f):
    #generate random points in [-1, 1] x [-1, 1]
    data = np.random.uniform(-1, 1, (N, 2))

    #define target labels
    p1, p2 = f
    labels = np.array([np.sign(np.cross(p2 - p1, x - p1)) for x in data])

    return data, labels

def pla(data, labels):
    w = np.zeros(3)  #dtarting with the zero vector (for w and bias)
    iteration = 0
    while True:
        misclassified = []
        for i, x in enumerate(data):
            x_augmented = np.concatenate(([1], x))  #with bias term
            if np.sign(np.dot(w, x_augmented)) != labels[i]:
                misclassified.append(i)

        if not misclassified:
            break

        #pick a random misclassified point
        i = random.choice(misclassified)
        x_augmented = np.concatenate(([1], data[i]))  #with bias term
        w += labels[i] * x_augmented
        iteration += 1
    return w, iteration

def compute_error(f, w, data, labels):
    errors = 0
    for i, x in enumerate(data):
        x_augmented = np.concatenate(([1], x))  #with bias term
        if np.sign(np.dot(w, x_augmented)) != labels[i]:
            errors += 1
    return errors / len(data)

def svm(data, labels):
    N, d = data.shape
    #set up the quadratic programming problem
    P = np.eye(d + 1)  #identity matrix of size (d + 1)
    P[0, 0] = 0  #no penalty for the bias term
    q = np.zeros(d + 1)

    #with bias term (add a column of 1s)
    X_augmented = np.hstack((np.ones((N, 1)), data))  # N x (d+1)

    #create constraint matrix G, each row corresponds to a constraint for a point
    G = -np.multiply(labels[:, np.newaxis], X_augmented)  # N x (d+1), each row is -y_n * (1, x_n)

    #the vector h contains -1 for each constraint
    h = -np.ones(N)

    #solve the quadratic programming problem
    qp_solution = solvers.qp(matrix(P), matrix(q), matrix(G), matrix(h))
    w_svm = np.array(qp_solution['x']).flatten()

    #compute the number of support vectors
    count_sv = 0
    for i, x in enumerate(data):
        x_augmented = np.concatenate(([1], x))  #augmented input with bias term
        if abs(labels[i] * np.dot(w_svm, x_augmented) - 1) < 0.001:
            count_sv += 1

    return w_svm, count_sv

#run the experiment over multiple runs
def run_experiment(N, num_runs):
    total_sv = 0
    svm_wins = 0
    for _ in range(num_runs):
        f = generate_target_function()
        data, labels = generate_data(N, f)
        test_data, test_labels = generate_data(N, f)

        # PLA
        w_pla, _ = pla(data, labels)
        pla_error = compute_error(f, w_pla, test_data, test_labels)

        # SVM
        w_svm, num_sv = svm(data, labels)
        svm_error = compute_error(f, w_svm, test_data, test_labels)
        total_sv += num_sv
        #check which method has a better error rate
        if svm_error <= pla_error:
            svm_wins += 1
    avg_sv = total_sv / num_runs
    svm_percentage = (svm_wins / num_runs) * 100
    return svm_percentage, avg_sv

svm_percentage, avg_sv = run_experiment(10, 1000)
print(f"SVM wins {svm_percentage:.2f}% of the time")
print(f'avg_sv: {avg_sv}')


SVM wins 83.30% of the time
avg_sv: 2.558


In [ ]:
svm_percentage, avg_sv = run_experiment(10, 1000)
print(f"SVM wins {svm_percentage:.2f}% of the time")
print(f'avg_sv: {avg_sv}')

SVM wins 84.70% of the time
avg_sv: 2.584


In [ ]:
import numpy as np
from cvxopt import matrix, solvers
import random

# Helper functions
def generate_target_function():
    # Randomly pick two points
    p1 = np.random.uniform(-1, 1, 2)
    p2 = np.random.uniform(-1, 1, 2)
    slope = (p2[1] - p1[1]) / (p2[0] - p1[0]) if p1[0] != p2[0] else 0
    intercept = p1[1] - slope * p1[0]
    return lambda x: np.sign(x[1] - (slope * x[0] + intercept))

def generate_data(N, f):
    data = []
    while len(data) < N:
        x = np.random.uniform(-1, 1, 2)
        y = f(x)
        data.append((np.array([1, x[0], x[1]]), y))
    return data

def pla(X, y, max_iterations=1000):
    w = np.zeros(X.shape[1])
    for _ in range(max_iterations):
        predictions = np.sign(X.dot(w))
        misclassified = np.where(predictions != y)[0]
        if len(misclassified) == 0:
            break
        rand_misclassified = random.choice(misclassified)
        w += y[rand_misclassified] * X[rand_misclassified]
    return w

def calculate_disagreement(f, g, num_points=1000):
    disagreements = 0
    for _ in range(num_points):
        x = np.random.uniform(-1, 1, 2)
        f_label = f(x)
        g_label = np.sign(g([1, x[0], x[1]]))
        if f_label != g_label:
            disagreements += 1
    return disagreements / num_points

def svm(data):
    X = np.array([x for x, _ in data])
    y = np.array([y for _, y in data])

    # Set up SVM QP problem
    m, n = X.shape
    P = matrix(np.outer(y, y) * (X @ X.T))
    q = matrix(-np.ones(m))
    G = matrix(-np.eye(m))
    h = matrix(np.zeros(m))
    A = matrix(y.reshape(1, -1), tc='d')
    b = matrix(0.0)

    # Solve QP
    solvers.options['show_progress'] = False
    solution = solvers.qp(P, q, G, h, A, b)
    alphas = np.ravel(solution['x'])

    # Find support vectors
    support_vectors = alphas > 1e-5
    support_vector_indices = np.where(support_vectors)[0]
    support_vectors_count = len(support_vector_indices)

    # Compute weights and bias
    w = ((alphas * y)[:, None] * X).sum(axis=0)
    b = np.mean(y[support_vectors] - X[support_vectors] @ w)

    # Define decision function g_SVM
    def g_svm(x):
        return np.sign(np.dot(w, x) + b)

    return g_svm, support_vectors_count

def experiment(N=10, runs=1000):
    svm_wins = 0
    total_support_vectors = 0

    for _ in range(runs):
        f = generate_target_function()
        data = generate_data(N, f)

        # Separate X and y
        X = np.array([point[0] for point in data])
        y = np.array([point[1] for point in data])

        # Run PLA
        w_pla = pla(X, y)
        g_pla = lambda x: np.sign(np.dot(w_pla, x))
        pla_disagreement = calculate_disagreement(f, g_pla)

        # Run SVM
        g_svm, support_vectors_count = svm(data)
        svm_disagreement = calculate_disagreement(f, g_svm)
        total_support_vectors += support_vectors_count

        # Compare errors
        if svm_disagreement < pla_disagreement:
            svm_wins += 1

    svm_win_percentage = (svm_wins / runs) * 100
    average_support_vectors = total_support_vectors / runs
    print(f"SVM wins {svm_win_percentage}% of the time.")
    print(f"Average number of support vectors: {average_support_vectors}")

# Run the experiment
experiment()
experiment(N=100)


SVM wins 54.2% of the time.
Average number of support vectors: 2.58
SVM wins 56.00000000000001% of the time.
Average number of support vectors: 3.102
